# Query Notebook

for the MAMI Path Transparency Measurement Summer School

### Configuration and Environment Setup

Set up out environment to point to the correct instance of the PTO, and set up our API token and the observation set ID for the normalized data we uploaded, as well as for the analyzed data combining observations from all students:

In [1]:
baseurl = "https://summer.pto.mami-project.eu"
token = None
my_obset_id = "5"

Now import some things we'll need to interact with the PTO:

In [2]:
# PTO client
from ptoclient import *

# Pandas
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# pretty printing
from pprint import pprint

And some utility functions we'll use to work with the dataframes we retrieve from the PTO:

In [3]:
def pivot_condition_time(df, prefix, aspect, states):
    """
    Given a dataframe counts of conditions (states of one or more aspects) 
    over a time series, pivot to a time-indexed table with a column for 
    each state.
    
    """
    
    aspect_df = df[df['condition'].map(lambda x: x.startswith(".".join((prefix,aspect))))]
    
    total_df = aspect_df.groupby('time').sum().loc[:,["count"]]
    total_df.columns = ['total']
    total_df['time'] = total_df.index
    
    aspect_df = aspect_df.join(total_df, on="time", rsuffix="_")
    del(aspect_df['time_'])
    
    pivot_df = aspect_df.groupby('time').first().loc[:,['total']]
    for c in states:
        cseries = aspect_df[aspect_df['condition'] == ".".join((prefix,aspect,c))]
        cseries.index = cseries['time']
        cseries = cseries.loc[:,['count']]
        cseries.columns = [c]
        pivot_df = pd.concat([pivot_df, cseries], axis=1)
    
    return pivot_df.fillna(0)

### Create a client to access the PTO

The client object encapsulates a base URL and an API token, and can be used to access queries and observations on the given instance of the PTO.

In [4]:
c = PTOClient(baseurl, token)

### Access my observation set

Let's check metadata for the observation set we uploaded in the earlier part of the course:

In [7]:
s = c.retrieve_set(setid=0xb)
pprint(s.metadata())

{'__created': '2018-06-11T15:34:59Z',
 '__data': 'https://summer.pto.mami-project.eu/obs/b/data',
 '__link': 'https://summer.pto.mami-project.eu/obs/b',
 '__modified': '2018-06-11T15:34:59Z',
 '_analyzer': 'https://raw.githubusercontent.com/mami-project/pto3-ecn/master/ecn_normalizer/ecn_normalizer.json',
 '_conditions': ['ecn.ipmark.ect0.seen',
                 'ecn.negotiation.failed',
                 'pathspider.not_observed',
                 'pathspider.missed_flows',
                 'ecn.connectivity.broken',
                 'ecn.connectivity.transient',
                 'ecn.connectivity.works',
                 'ecn.negotiation.succeeded',
                 'ecn.ipmark.ect1.not_seen',
                 'ecn.ipmark.ect0.not_seen',
                 'ecn.ipmark.ce.not_seen',
                 'ecn.negotiation.reflected',
                 'ecn.ipmark.ce.seen',
                 'ecn.connectivity.offline'],
 '_owner': 'laurent.schumacher@ieee.org',
 '_sources': ['https://summer.pto.m

Now run a query to look at the ratio of ECN negotiation success in our observation set:

In [23]:
q_mine = c.submit_query(PTOQuerySpec().time("2018-06-10", "2018-06-12").set_id(0x6)
                                      .condition("ecn.negotiation.*")
                                      .group_by_condition())

Retrieve metadata, wait for state to be complete:

In [24]:
pprint(q_mine.metadata(reload=True))

{'__completed': '2018-06-11T17:39:16+02:00',
 '__created': '2018-06-11T17:39:16+02:00',
 '__encoded': 'time_start=2018-06-10T00%3A00%3A00Z&time_end=2018-06-12T00%3A00%3A00Z&set=6&condition=ecn.negotiation.failed&condition=ecn.negotiation.reflected&condition=ecn.negotiation.succeeded&group=condition',
 '__executed': '2018-06-11T17:39:16+02:00',
 '__link': 'https://summer.pto.mami-project.eu/query/20175c902a12cbddd084a3728a500a91a86e98e324ac9682cc1b06ce90ab07d8',
 '__modified': '2018-06-11T17:39:16+02:00',
 '__result': 'https://summer.pto.mami-project.eu/query/20175c902a12cbddd084a3728a500a91a86e98e324ac9682cc1b06ce90ab07d8/result',
 '__state': 'complete'}


Have a look at the query results, then calculate the ratio of targets where ECN negotiation succeeded:

In [25]:
r = q_mine.results()
r

,count,group
0,6489,ecn.negotiation.succeeded


In [ ]:
r.index = r.group
r.loc['ecn.negotiation.succeeded']['count'] / r['count'].sum()

### Examine combined results

Let's run another query, but on all multipoint conditions -- these look at the result of analysis of each target combining observations from each student:

In [20]:
q_multi = c.submit_query(PTOQuerySpec().time("2018-06-10", "2018-06-12").set_id(0xc)
                                       .condition("ecn.multipoint.*")
                                       .group_by_condition())

In [21]:
pprint(q_multi.metadata(reload=True))

{'__completed': '2018-06-11T17:41:17+02:00',
 '__created': '2018-06-11T17:41:17+02:00',
 '__encoded': 'time_start=2018-06-10T00%3A00%3A00Z&time_end=2018-06-12T00%3A00%3A00Z&set=12&condition=ecn.multipoint.connectivity.broken&condition=ecn.multipoint.connectivity.offline&condition=ecn.multipoint.connectivity.path_dependent&condition=ecn.multipoint.connectivity.transient&condition=ecn.multipoint.connectivity.unstable&condition=ecn.multipoint.connectivity.works&condition=ecn.multipoint.negotiation.failed&condition=ecn.multipoint.negotiation.path_dependent&condition=ecn.multipoint.negotiation.succeeded&condition=ecn.multipoint.negotiation.unstable&group=condition',
 '__executed': '2018-06-11T17:41:17+02:00',
 '__link': 'https://summer.pto.mami-project.eu/query/e9ea2ffb2e66b51a01625fac5cf68cee3841ce55c49d2ab833be0d7467d04e32',
 '__modified': '2018-06-11T17:41:17+02:00',
 '__result': 'https://summer.pto.mami-project.eu/query/e9ea2ffb2e66b51a01625fac5cf68cee3841ce55c49d2ab833be0d7467d04e32/re

In [22]:
r = q_multi.results()
r

,count,group
0,17,ecn.multipoint.connectivity.broken
1,177,ecn.multipoint.connectivity.offline
2,148,ecn.multipoint.connectivity.path_dependent
3,28,ecn.multipoint.connectivity.transient
4,171,ecn.multipoint.connectivity.unstable
5,8087,ecn.multipoint.connectivity.works
6,182,ecn.multipoint.negotiation.failed
7,1663,ecn.multipoint.negotiation.path_dependent
8,6561,ecn.multipoint.negotiation.succeeded
9,222,ecn.multipoint.negotiation.unstable


In [32]:
q_pathdep_obs = c.submit_query(PTOQuerySpec().time("2018-06-10", "2018-06-12").set_id(0xc)
                                             .condition("ecn.multipoint.negotiation.path_dependent"))

In [33]:
pprint(q_pathdep_obs.metadata(reload=True))

{'__completed': '2018-06-11T17:45:57+02:00',
 '__created': '2018-06-11T17:45:57+02:00',
 '__encoded': 'time_start=2018-06-10T00%3A00%3A00Z&time_end=2018-06-12T00%3A00%3A00Z&set=12&condition=ecn.multipoint.negotiation.path_dependent',
 '__executed': '2018-06-11T17:45:57+02:00',
 '__link': 'https://summer.pto.mami-project.eu/query/c88cf925688eacbba159fc5de4548baa029066a63b6361bbc48504442204e056',
 '__modified': '2018-06-11T17:45:57+02:00',
 '__result': 'https://summer.pto.mami-project.eu/query/c88cf925688eacbba159fc5de4548baa029066a63b6361bbc48504442204e056/result',
 '__state': 'complete'}


In [34]:
r = q_pathdep_obs.results()
r

,condition,path,set_id,time_end,time_start,value
0,ecn.multipoint.negotiation.path_dependent,* 151.101.65.69,c,2018-06-11T14:12:35Z,2018-06-11T14:12:34Z,6
1,ecn.multipoint.negotiation.path_dependent,* 64.233.167.139,c,2018-06-11T13:18:42Z,2018-06-11T13:18:37Z,6
2,ecn.multipoint.negotiation.path_dependent,* 104.16.166.50,c,2018-06-11T13:12:11Z,2018-06-11T13:12:11Z,6
3,ecn.multipoint.negotiation.path_dependent,* 209.225.49.78,c,2018-06-11T13:14:25Z,2018-06-11T13:14:15Z,6
4,ecn.multipoint.negotiation.path_dependent,* 210.162.157.12,c,2018-06-11T13:33:00Z,2018-06-11T13:32:55Z,4
5,ecn.multipoint.negotiation.path_dependent,* 104.16.115.22,c,2018-06-11T13:13:50Z,2018-06-11T13:13:50Z,6
6,ecn.multipoint.negotiation.path_dependent,* 198.49.23.177,c,2018-06-11T13:23:20Z,2018-06-11T13:23:15Z,5
7,ecn.multipoint.negotiation.path_dependent,* 103.42.78.176,c,2018-06-11T13:39:36Z,2018-06-11T13:39:30Z,4
8,ecn.multipoint.negotiation.path_dependent,* 156.54.106.174,c,2018-06-11T13:27:59Z,2018-06-11T13:27:59Z,5
9,ecn.multipoint.negotiation.path_dependent,* 217.175.155.120,c,2018-06-11T13:35:13Z,2018-06-11T13:35:09Z,4


In [35]:
q_specific = c.submit_query(PTOQuerySpec().time("2018-06-10", "2018-06-12")
                                             .target("170.218.212.80"))

In [37]:
pprint(q_specific.metadata(reload=True))

{'__completed': '2018-06-11T17:47:07+02:00',
 '__created': '2018-06-11T17:47:02+02:00',
 '__encoded': 'time_start=2018-06-10T00%3A00%3A00Z&time_end=2018-06-12T00%3A00%3A00Z&target=170.218.212.80',
 '__executed': '2018-06-11T17:47:02+02:00',
 '__link': 'https://summer.pto.mami-project.eu/query/8ca796cd0656b59fa20956ec29af7de9307fa6f0c9cb9b8ee824ba4a1a512503',
 '__modified': '2018-06-11T17:47:07+02:00',
 '__result': 'https://summer.pto.mami-project.eu/query/8ca796cd0656b59fa20956ec29af7de9307fa6f0c9cb9b8ee824ba4a1a512503/result',
 '__state': 'complete'}


In [38]:
r = q_specific.results()
r

,condition,path,set_id,time_end,time_start,value
0,ecn.connectivity.works,137.50.216.187 AS786 * AS11740 170.218.212.80,6,2018-06-11T13:38:18Z,2018-06-11T13:38:14Z,0
1,ecn.negotiation.succeeded,137.50.216.187 AS786 * AS11740 170.218.212.80,6,2018-06-11T13:38:18Z,2018-06-11T13:38:14Z,0
2,ecn.ipmark.ect0.seen,137.50.216.187 AS786 * AS11740 170.218.212.80,6,2018-06-11T13:38:18Z,2018-06-11T13:38:14Z,0
3,ecn.ipmark.ect1.not_seen,137.50.216.187 AS786 * AS11740 170.218.212.80,6,2018-06-11T13:38:18Z,2018-06-11T13:38:14Z,0
4,ecn.ipmark.ce.not_seen,137.50.216.187 AS786 * AS11740 170.218.212.80,6,2018-06-11T13:38:18Z,2018-06-11T13:38:14Z,0
5,ecn.connectivity.works,137.50.18.151 AS786 * AS11740 170.218.212.80,7,2018-06-11T13:46:09Z,2018-06-11T13:46:07Z,0
6,ecn.negotiation.failed,137.50.18.151 AS786 * AS11740 170.218.212.80,7,2018-06-11T13:46:09Z,2018-06-11T13:46:07Z,0
7,ecn.ipmark.ect0.not_seen,137.50.18.151 AS786 * AS11740 170.218.212.80,7,2018-06-11T13:46:09Z,2018-06-11T13:46:07Z,0
8,ecn.ipmark.ect1.not_seen,137.50.18.151 AS786 * AS11740 170.218.212.80,7,2018-06-11T13:46:09Z,2018-06-11T13:46:07Z,0
9,ecn.ipmark.ce.not_seen,137.50.18.151 AS786 * AS11740 170.218.212.80,7,2018-06-11T13:46:09Z,2018-06-11T13:46:07Z,0
